In [3]:
import numpy as np

class SudokuBoard():
    def __init__(self):
        self.board=np.zeros((9,9),np.int64)
    def set(self,row,col,value):
        if row>=0 and row<9 and col>=0 and col<9:
            self.board[row,col]=value
            return True
        return False
    def set_all(self,values):
        for value in values:
            self.set(value[0],value[1],value[2])
    def __str__(self):
        rows = []
        rows.append('#+------+-------+------+')
        # passa por cada linha pegando numeros ou vazios
        for y in range(9):
            row=[]
            for e in self.board[y,:]: # pega elementos da linha
                if e==0:
                    row.append(' ')
                else:
                    row.append(str(e))
            # adiciona sepadores verticais
            row.insert(3, '|')
            row.insert(7, '|')
            rows.append('#|'+' '.join(row)+'|') # coloca nas linhas separando cada elemento com um espaco
            # Add a horizontal separator, if needed.
            if y == 2 or y == 5 or y==8:
                rows.append('#+------+-------+------+')

        return '\n'.join(rows)        

    def solve(self):
        """ 
            Retorna um board com a solucao respeitando os numeros 
            colocados inicialmente via set ou set_all
        """
        pass

board=SudokuBoard()

board.set_all([
    [0,0,2],[0,1,5],[0,5,3],[0,7,9],[0,8,1],
    [1,1,1]
])
board.set(1,5,4)
print(board)

#+------+-------+------+
#|2 5   |     3 |   9 1|
#|  1   |     4 |      |
#|      |       |      |
#+------+-------+------+
#|      |       |      |
#|      |       |      |
#|      |       |      |
#+------+-------+------+
#|      |       |      |
#|      |       |      |
#|      |       |      |
#+------+-------+------+
